In [1]:
import pandas as pd
import numpy as np
import math, time
import tensorflow
import cv2
from keras.models import load_model
from time import time
from datetime import datetime
from PIL import Image
from IPython.display import display, HTML
from tqdm import tqdm
import itertools as it
from IPython.display import display, HTML
from matplotlib import pyplot as plt
from scipy.stats import mode
import warnings 
warnings.filterwarnings("ignore")


filename="ITMD 525_03.MOV"   #filename
cap = cv2.VideoCapture(filename) #Read Frame
fps=cap.get(cv2.CAP_PROP_FPS) #Extract the frame per second (fps) 
print('Frames per second : ', fps,'FPS')
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)#get overall no# of frames in the video file
print('Frame count : ', frame_count)
sessions=pd.read_excel('session_info.xlsx', sheet_name='Sheet2')
sessions=sessions.rename(columns=lambda x:x.strip())
ses_crop=sessions.iloc[:, 0:7] #cropping the session info table for columns of interest 
display(HTML(ses_crop.to_html()))

#selecting only sessions for which we have homework values 
ses_crop=ses_crop[np.isnan(ses_crop['# of students with wrong answers'])==False]
ses_crop.reset_index(drop=True, inplace=True)
display(HTML(ses_crop.to_html()))

AttributeError: module 'numpy' has no attribute 'typeDict'

C:\ProgramData\Anaconda3\envs\cv_open\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


Frames per second :  29.97002997002997 FPS
Frame count :  93268.0


,File Name,Start time,End time,session_name,# of students in video,which assignments,# of students with wrong answers
0,ITMD 525_03.MOV,6.25,9.00,Information Overload & Solutions for Information Overload,13,NaN,NaN
1,ITMD 525_03.MOV,9.22,10.09,Chunking,13,NaN,NaN
2,ITMD 525_03.MOV,10.10,10.29,Omission,13,NaN,NaN
3,ITMD 525_03.MOV,10.30,10.51,Queuing,13,NaN,NaN
4,ITMD 525_03.MOV,10.52,11.17,Capitulation,16,NaN,NaN
5,ITMD 525_03.MOV,11.40,12.19,Filtering & Giving examples about who uses RecSys,16,NaN,NaN
6,ITMD 525_03.MOV,16.20,19.45,RecSys,17,NaN,NaN
7,ITMD 525_03.MOV,19.46,22.14,How RecSys works (Netflix Example),18,NaN,NaN
8,ITMD 525_03.MOV,22.15,24.20,How it works,18,NaN,NaN
9,ITMD 525_03.MOV,24.22,28.07,Traditional Recommendation Algs,18,NaN,NaN


,File Name,Start time,End time,session_name,# of students in video,which assignments,# of students with wrong answers
0,ITMD 525_03.MOV,36.50,40.29,Rating Prediction,18,hw6,8.0
1,ITMD 525_03.MOV,40.29,46.13,Top-N Recommendation,18,hw 6,3.0
2,ITMD 525_03.MOV,47.26,51.52,Relevance Metrics,18,"hw2, hw6",3.0


In [2]:
#let's convert the start time and end time columns into proper time format i.e "%H:%M:%S"
import math, time
starttime=[]
for i in range(len(ses_crop)):
    starttime.append(math.modf(ses_crop['Start time'][i])) # (0.5678000000000338, 1234.0)
print('\nstart time in disected form:', starttime)
    
seconds_s=[round(x[0]*100) for x in starttime]
secondsfromminutes_s=[x[1]*60 for x in starttime]

total_seconds_s=[]
for i in range(len(seconds_s)):
    total_seconds_s.append(seconds_s[i]+secondsfromminutes_s[i])
Start_time=[]
for i in range(len(total_seconds_s)):
    Start_time.append(time.strftime("%H:%M:%S", time.gmtime(total_seconds_s[i])))
print('\n\nStart time in Time Format:',Start_time)

#doing the same for End time
endtime=[]
for i in range(len(ses_crop)):
    endtime.append(math.modf(ses_crop['End time'][i])) # (0.5678000000000338, 1234.0)
print('\nEnd time in disected form:',endtime)

seconds_e=[round(x[0]*100) for x in endtime]
secondsfromminutes_e=[x[1]*60 for x in endtime]

total_seconds_e=[]
for i in range(len(seconds_e)):
    total_seconds_e.append(seconds_e[i]+secondsfromminutes_e[i])
End_time=[]
for i in range(len(total_seconds_e)):
    End_time.append(time.strftime("%H:%M:%S", time.gmtime(total_seconds_e[i])))
print('\n\nEnd time in Time Format:',End_time)

ses_len=[]
for i in range(len(total_seconds_s)):
    ses_len.append(total_seconds_e[i]-total_seconds_s[i])


start time in disected form: [(0.5, 36.0), (0.28999999999999915, 40.0), (0.259999999999998, 47.0)]


Start time in Time Format: ['00:36:50', '00:40:29', '00:47:26']

End time in disected form: [(0.28999999999999915, 40.0), (0.13000000000000256, 46.0), (0.5200000000000031, 51.0)]


End time in Time Format: ['00:40:29', '00:46:13', '00:51:52']


In [3]:
#assigning the time values in correct format to cropped session dataframe
ses_crop['Start time']=Start_time
ses_crop['End time']=End_time
display(HTML(ses_crop.to_html()))

#slicing session-wise attendance for comparison at later stage
num_students=ses_crop.iloc[:,3:5]
display(HTML(num_students.to_html()))


,File Name,Start time,End time,session_name,# of students in video,which assignments,# of students with wrong answers
0,ITMD 525_03.MOV,00:36:50,00:40:29,Rating Prediction,18,hw6,8.0
1,ITMD 525_03.MOV,00:40:29,00:46:13,Top-N Recommendation,18,hw 6,3.0
2,ITMD 525_03.MOV,00:47:26,00:51:52,Relevance Metrics,18,"hw2, hw6",3.0


,session_name,# of students in video
0,Rating Prediction,18
1,Top-N Recommendation,18
2,Relevance Metrics,18


In [4]:
#for each session we have to divide the video into 3 slices begin, dom, end
#for each of sub-sessions, we have to determine start frame and end frame for slicing the sessions
#let's determine startframe and endframe for begin session
startframe_begin=[]
for i in range(len(total_seconds_s)):
    startframe_begin.append(fps*(total_seconds_s[i]))

#after 5 seconds, begin subsession ends, therefore end frame of the begin subsession is located at (total_seconds_ofstarttime+5 seconds)*fps
endframe_begin=[]
for i in range(len(total_seconds_s)):
    endframe_begin.append(fps*(total_seconds_s[i]+5))

#endframe_begin = startframe_dom
#endframe_dom = startframe_end

#startframe_end & endframe_end can be calculated with by using total_seconds_e
startframe_end=[]
for i in range(len(total_seconds_e)):
    startframe_end.append(fps*(total_seconds_e[i]-5))
    
endframe_end=[]
for i in range(len(total_seconds_e)):
    endframe_end.append(fps*total_seconds_e[i])

my_dict={'session_name':ses_crop['session_name'],'startframe_begin':startframe_begin, 'endframe_begin':endframe_begin,'startframe_end':startframe_end,'endframe_end':endframe_end, 'session_length':ses_len}
ses_timeframes=pd.DataFrame(my_dict)


#reseting the index so that loop iteration does not face any probelms
ses_timeframes.reset_index(drop=True, inplace=True)
display(HTML(ses_timeframes.to_html()))

,session_name,startframe_begin,endframe_begin,startframe_end,endframe_end,session_length
0,Rating Prediction,66233.766234,66383.616384,72647.352647,72797.202797,219.0
1,Top-N Recommendation,72797.202797,72947.052947,82957.042957,83106.893107,344.0
2,Relevance Metrics,85294.705295,85444.555445,93116.883117,93266.733267,266.0


In [5]:
#let's now break down the video files into sessions as given in excel file
#further each session is going to be further divided into sub-sessions: begin, dom, end

tic=time.time()

#initializing lists for storing session-wise values for each subsession
begin_list = []
dom_list = []
end_list =[]
useless=[]
c =0

#setting height and width of the frame to be stored for face recognition
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #height
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #width

for i in tqdm(range(int(frame_count))):
    #following if-elif statements are only meants for determining value of c for iterating through the ses_timeframes table shown above
    
    if 66233.766234  <= i <= 72797.202797:
        c=0
    elif 72797.202797  <= i <= 83106.893107:
        c=1
    elif 85294.705295  <= i <= 93266.733267:
        c=2
  
       
    #read the frame from given video file
    ret, frame = cap.read()
    
    frame=cv2.resize(frame, (width,height))  #resize the frame
    
    if i>=ses_timeframes['startframe_begin'][c] and i<=ses_timeframes['endframe_begin'][c]:  
        begin_list.append([ses_timeframes['session_name'][c],frame])            #output
    elif i>=ses_timeframes['endframe_begin'][c] and i<=ses_timeframes['startframe_end'][c]:
        dom_list.append([ses_timeframes['session_name'][c],frame]) 
    elif i>=ses_timeframes['startframe_end'][c] and i<=ses_timeframes['endframe_end'][c]:  
        end_list.append([ses_timeframes['session_name'][c],frame]) 
   # else:
    #    useless.append(frame)
        
#creating data frames out of lists
df_begin = pd.DataFrame(begin_list, columns=["session_name", "frame"])
df_dom = pd.DataFrame(dom_list, columns=["session_name", "frame"])
df_end = pd.DataFrame(end_list, columns=["session_name", "frame"]) 

#printing heads for begin, dom and end dfs
print('Begin Sessions Number of Frames', len(df_begin),  df_begin.head())
print('\n\nDOM Sessions Number of Frames', len(df_dom), df_dom.head())
print('\n\nEnd Sessions Number of Frames', len(df_end), df_end.head())  

tac=time.time()
print('Execution Time: ',round(tac-tic),' sec')


100%|██████████████████████████████████████████████████████████████████████████| 93268/93268 [00:56<00:00, 1643.03it/s]


Begin Sessions Number of Frames 450         session_name                                              frame
0  Rating Prediction  [[[72, 133, 147], [72, 133, 147], [72, 133, 14...
1  Rating Prediction  [[[72, 133, 147], [72, 133, 147], [72, 133, 14...
2  Rating Prediction  [[[73, 134, 148], [73, 134, 148], [73, 134, 14...
3  Rating Prediction  [[[71, 134, 148], [71, 134, 148], [71, 134, 14...
4  Rating Prediction  [[[73, 134, 148], [73, 134, 148], [73, 134, 14...


DOM Sessions Number of Frames 23946         session_name                                              frame
0  Rating Prediction  [[[77, 139, 151], [77, 139, 151], [77, 139, 15...
1  Rating Prediction  [[[76, 138, 150], [76, 138, 150], [76, 138, 15...
2  Rating Prediction  [[[76, 138, 150], [76, 138, 150], [76, 138, 15...
3  Rating Prediction  [[[76, 138, 150], [76, 138, 150], [76, 138, 15...
4  Rating Prediction  [[[74, 138, 150], [74, 138, 150], [74, 138, 15...


End Sessions Number of Frames 449         session_name      

In [6]:
#just checking the number of frames per subsession 
begin_sessions=df_begin.groupby(['session_name'], sort=False).count()
display(HTML(begin_sessions.to_html()))
dom_sessions=df_dom.groupby(['session_name'],sort=False).count()
display(HTML(dom_sessions.to_html()))
end_sessions=df_end.groupby(['session_name'],sort=False).count()
display(HTML(end_sessions.to_html()))

,frame
session_name,
Rating Prediction,150
Top-N Recommendation,150
Relevance Metrics,150


,frame
session_name,
Rating Prediction,6264
Top-N Recommendation,10010
Relevance Metrics,7672


,frame
session_name,
Rating Prediction,150
Top-N Recommendation,149
Relevance Metrics,150


In [9]:
#function for applying HaarCascade algorithm on every
def NumFaces_Haar (image):
    #changing color model for the input image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces using the classifier
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    #number of faces detected by the model
    num_faces = len(faces)
    return num_faces

    

In [10]:
#let's apply Haar Cascade for face detection

# Load pre-trained classifiers
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

#detect number of faces for each frame of begin subsessions
face_list_begin=[]
for i in tqdm(range(len(df_begin))):
    frame_1=df_begin['frame'][i]
    num_faces=NumFaces_Haar(frame_1)
    face_list_begin.append([df_begin['session_name'][i], num_faces])
          
faces_begin_hog = pd.DataFrame(face_list_begin, columns=["session_name", "no_of_Faces"])

#detect number of faces for each frame of dom subsessions
face_list_dom=[]
for i in tqdm(range(len(df_dom))):
    frame_1=df_dom['frame'][i]
    num_faces=NumFaces_Haar(frame_1)
    face_list_dom.append([df_dom['session_name'][i], num_faces])
          
faces_dom_hog = pd.DataFrame(face_list_dom, columns=["session_name", "no_of_Faces"])

#detect number of faces for each frame of end subsessions
face_list_end=[]
for i in tqdm(range(len(df_end))):
    frame_1=df_end['frame'][i]
    num_faces=NumFaces_Haar(frame_1)
    face_list_end.append([df_end['session_name'][i], num_faces])
          
faces_end_hog = pd.DataFrame(face_list_end, columns=["session_name", "no_of_Faces"])

#compute MINIMUM of number detected faces in each subsession
face_begin_min_hog=faces_begin_hog.groupby(['session_name'],sort=False).min()
face_dom_min_hog=faces_dom_hog.groupby(['session_name'],sort=False).min()
face_end_min_hog=faces_end_hog.groupby(['session_name'],sort=False).min()
#compute MAXIMUM of number detected faces in each subsession
face_begin_max_hog=faces_begin_hog.groupby(['session_name'],sort=False).max()
face_dom_max_hog=faces_dom_hog.groupby(['session_name'],sort=False).max()
face_end_max_hog=faces_end_hog.groupby(['session_name'],sort=False).max()
#compute AVERAGE of number detected faces in each subsession
face_begin_avg_hog=faces_begin_hog.groupby(['session_name'],sort=False).mean()
face_dom_avg_hog=faces_dom_hog.groupby(['session_name'],sort=False).mean()
face_end_avg_hog=faces_end_hog.groupby(['session_name'],sort=False).mean()

100%|████████████████████████████████████████████████████████████████████████████████| 449/449 [00:10<00:00, 42.19it/s]


In [11]:


#let's analyze variation in [minimum, average, and maximum] number of faces detected in the begin session 
merged_2_begin_hog=face_begin_min_hog.merge(face_begin_avg_hog, on='session_name', how='outer')
merged_all_begin_hog=merged_2_begin_hog.merge(face_begin_max_hog, on='session_name',how='outer')
num_students_begin_hog=merged_all_begin_hog.merge(num_students, on='session_name',how='outer')
num_students_begin_hog.columns = ['begin session name', 'minimum', 'average', 'maximum', 'actual']
display(HTML(num_students_begin_hog.to_html()))

#let's analyze variation in [minimum, average, and maximum] number of faces detected in the dominant session 
merged_2_dom_hog=face_dom_min_hog.merge(face_dom_avg_hog, on='session_name', how='outer')
merged_all_dom_hog=merged_2_dom_hog.merge(face_dom_max_hog, on='session_name',how='outer')
num_students_dom_hog=merged_all_dom_hog.merge(num_students, on='session_name',how='outer')
num_students_dom_hog.columns = ['begin session name', 'minimum', 'average', 'maximum', 'actual']
display(HTML(num_students_dom_hog.to_html()))

#let's analyze variation in [minimum, average, and maximum] number of faces detected in the dominant session 
merged_2_end_hog=face_end_min_hog.merge(face_end_avg_hog, on='session_name', how='outer')
merged_all_end_hog=merged_2_end_hog.merge(face_end_max_hog, on='session_name',how='outer')
num_students_end_hog=merged_all_end_hog.merge(num_students, on='session_name',how='outer')
num_students_begin_hog.columns = ['begin session name', 'minimum', 'average', 'maximum', 'actual']
num_students_end_hog.columns = ['begin session name', 'minimum', 'average', 'maximum', 'actual']
display(HTML(num_students_end_hog.to_html()))

,begin session name,minimum,average,maximum,actual
0,Rating Prediction,1,1.346667,2,18
1,Top-N Recommendation,1,1.460000,2,18
2,Relevance Metrics,1,1.460000,2,18


,begin session name,minimum,average,maximum,actual
0,Rating Prediction,0,1.232280,3,18
1,Top-N Recommendation,0,1.022078,3,18
2,Relevance Metrics,0,0.731491,3,18


,begin session name,minimum,average,maximum,actual
0,Rating Prediction,1,1.253333,2,18
1,Top-N Recommendation,0,1.718121,3,18
2,Relevance Metrics,0,1.173333,2,18
